<img src="Images/PU.png" width="100%">

## Course Name : AI 702 Deep Learning for Professionals  
#### Notebook compiled by : Bhushan Garware, Project Lead at Learning and Development  
** Important ! ** For internal circulation olny

In [1]:
import time
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, SpatialDropout1D
from keras.layers import LSTM

Using TensorFlow backend.


In [2]:
# training:
epochs = 4
batch_size = 128

# vector-space embedding: 
n_dim = 64 
n_unique_words = 10000 
max_review_length = 100 
pad_type = trunc_type = 'pre'
drop_embed = 0.2 

# LSTM layer architecture:
n_lstm = 256 
drop_lstm = 0.2


<img src="Images/LSTM_Network.png" width="100%">

In [3]:
(x_train, y_train), (x_valid, y_valid) = imdb.load_data(num_words=n_unique_words) 

In [4]:
x_train = pad_sequences(x_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
x_valid = pad_sequences(x_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

In [5]:
model = Sequential()
model.add(Embedding(n_unique_words, n_dim, input_length=max_review_length)) 
model.add(SpatialDropout1D(drop_embed))
model.add(LSTM(n_lstm, dropout=drop_lstm))
model.add(Dense(1, activation='sigmoid'))

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 64)           640000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 100, 64)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               328704    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 968,961
Trainable params: 968,961
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Model training below fails with ResourceExhaustedError on AWS EC2 instance. Need to find out why..

In [8]:
start_model_training = time.time()

In [9]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_valid, y_valid))

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 329s 13ms/step - loss: 0.5408 - acc: 0.7181 - val_loss: 0.3729 - val_acc: 0.8284
Epoch 2/4
25000/25000 [==============================] - 292s 12ms/step - loss: 0.3047 - acc: 0.8715 - val_loss: 0.3470 - val_acc: 0.8483
Epoch 3/4
25000/25000 [==============================] - 363s 15ms/step - loss: 0.2395 - acc: 0.9047 - val_loss: 0.3778 - val_acc: 0.8431
Epoch 4/4
25000/25000 [==============================] - 280s 11ms/step - loss: 0.2014 - acc: 0.9232 - val_loss: 0.3775 - val_acc: 0.8440


In [10]:
end_model_training = time.time()
print("Process took seconds:",(end_model_training-start_model_training))

Process took seconds: 1267.0280947685242


In [ ]:
y_hat = model.predict_proba(x_valid)

In [ ]:
y_hat

In [ ]:
y_valid

In [ ]:
float_y_hat = []
for y in y_hat:
    float_y_hat.append(y[0])

In [ ]:
import pandas as pd

In [ ]:
# print predicted and actual to get general idea on model performance
# BTW, zip creates a new list, filled with tuples containing elements from the iterable arguments
ydf = pd.DataFrame(list(zip(float_y_hat, y_valid)), columns=['y_hat', 'y'])
ydf.head(10)

In [ ]:
ydf.shape

In [ ]:
# print wrong predictions
ydf[(ydf.y == 0) & (ydf.y_hat > 0.9)].head(10)

In [ ]:
# Check how many wrong predictions for class 0?
ydf[(ydf.y == 0) & (ydf.y_hat > 0.6)].shape

In [ ]:
# Check how many wrong predictions for class 1?
ydf[(ydf.y == 1) & (ydf.y_hat < 0.4)].shape